# 1. Preprocessing

## 1-a. Import Data and libraries

In [2]:
#Loading library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time
import numpy as np

In [3]:
df = pd.read_csv("cleaned_data.csv",index_col=0)
df = df.drop('time',axis=1)
df.head()

,category,viewCount,likeCount,commentCount,duration,made_for_kids,viewPerComment,tagsCount,year,weekday,month,time_of_day,title_length
0,5,2,39271,2538,4,0,165.401103,8,2023,0,5,0,18
1,5,4,156694,6633,3,0,364.847128,6,2023,6,5,0,35
2,5,4,78836,2315,5,0,742.543413,3,2023,5,5,1,34
3,5,4,119458,5293,2,0,390.751181,9,2023,4,5,0,30
4,5,4,113299,4649,2,0,440.775436,9,2023,3,5,1,16


In [4]:
df.describe()

,category,viewCount,likeCount,commentCount,duration,made_for_kids,viewPerComment,tagsCount,year,weekday,month,time_of_day,title_length
count,34621.000000,34621.000000,3.462100e+04,34621.000000,34621.000000,34621.000000,34621.000000,34621.000000,34621.000000,34621.000000,34621.000000,34621.000000,34621.000000
mean,6.226972,3.089310,7.358566e+04,4437.750296,1.826926,0.000289,883.064358,15.499928,2017.403223,2.958378,6.479160,0.827821,50.488721
std,3.528896,1.501606,2.058486e+05,10732.664863,1.433032,0.016993,1395.020067,11.285734,3.458414,1.949366,3.455963,0.611762,22.123522
min,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,2008.000000,0.000000,1.000000,0.000000,1.000000
25%,5.000000,2.000000,4.579000e+03,337.000000,1.000000,0.000000,313.686341,7.000000,2015.000000,1.000000,3.000000,0.000000,35.000000
50%,5.000000,3.000000,2.400400e+04,1487.000000,2.000000,0.000000,540.695203,14.000000,2018.000000,3.000000,6.000000,1.000000,48.000000
75%,7.000000,4.000000,7.497000e+04,4660.000000,2.000000,0.000000,1011.403274,23.000000,2020.000000,5.000000,9.000000,1.000000,64.000000
max,14.000000,9.000000,1.219854e+07,617657.000000,5.000000,1.000000,89817.454545,83.000000,2023.000000,6.000000,12.000000,2.000000,101.000000


#### KeyTable for references:

1. Category
![image.png](attachment:image.png)

2. Duration

![image-3.png](attachment:image-3.png)

3. View Counts (Response Variable)

![image-2.png](attachment:image-2.png)

## 1-b. Train, test split

In [6]:
#split the data into train and test set

from sklearn.model_selection import train_test_split

np.random.seed(1111)
raw_train, raw_test = train_test_split(df, test_size = 0.2)

In [8]:
X = df.drop(['viewCount'], axis = 1)
y = df['viewCount']

X_train, X_test, y_train,  y_test  = train_test_split(X, y, test_size = 0.2)

In [6]:
y_train.shape

(27696,)

In [9]:
X_train

,category,likeCount,commentCount,duration,made_for_kids,viewPerComment,tagsCount,year,weekday,month,time_of_day,title_length
30735,3,1979,817,2,0,151.157895,27,2020,1,8,1,96
9957,5,7768,268,5,0,1254.167910,20,2017,5,12,1,68
25156,1,36242,2948,2,0,813.363976,38,2018,3,9,2,62
17713,12,1430,381,1,0,50.144357,0,2013,2,9,0,36
16289,12,34432,1915,1,0,874.626110,13,2016,5,11,1,45
...,...,...,...,...,...,...,...,...,...,...,...,...
2374,5,84648,8158,2,0,400.011890,24,2016,1,5,1,5
32682,3,489,39,1,0,683.384615,26,2017,4,3,1,89
6066,3,295356,8281,2,0,1289.378819,6,2018,6,11,1,39
25250,1,35678,3664,1,0,802.075328,47,2017,2,6,2,20


## 1-c. Scaling data

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)


# 2. Model Implementation

## 2-a. Random Forest

**Initial Model**

In [8]:
# Initial Model 
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=300, 
                             max_depth=3)

# Fit RandomForestClassifier
RF.fit(X_train, y_train)
# Predict the test set labels
y_pred = RF.predict(X_test)

In [9]:
print("Test score: ",RF.score(X_test, y_test))
print("Training score: ", RF.score(X_train, y_train))

Test score:  0.6462093862815884
Training score:  0.6547876949740035


**Hyperparameter tuning with cross validation**

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Hyperameter grid
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [3, 6, 9, 12, None]
}

RF = RandomForestClassifier()
grid_search = GridSearchCV(RF, param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_RF = grid_search.best_estimator_
y_pred = best_RF.predict(X_test)

# Evaluate
test_score = best_RF.score(X_test, y_test)
train_score = best_RF.score(X_train, y_train)

print("Test score: ", test_score)
print("Training score: ", train_score)
print("Best parameters: ", grid_search.best_params_)


C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


KeyboardInterrupt: 

**Random Forest Final Model**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators = 300, 
                             max_depth = None)

RF.fit(X_train, y_train)
y_pred = RF.predict(X_test)
rf_accuracy_test = RF.score(X_test, y_test)
rf_accuracy_train = RF.score(X_train, y_train)

In [ ]:
print("Random Forest Test score: ",rf_accuracy_test)
print("Random Forest Training score: ", rf_accuracy_train)

In [ ]:
## Check precision, recall and F1 score
from sklearn.metrics import precision_score, recall_score, f1_score

rf_precision = precision_score(y_test, y_pred, average='weighted')
rf_recall = recall_score(y_test, y_pred, average='weighted')
rf_f1 = f1_score(y_test, y_pred, average='weighted')


print("Random Forest Precision:", rf_precision)
print("Random Forest Recall:", rf_recall)
print("Random Forest F1 score:", rf_f1)


- The precision of the Random Forest model is about 0.9198, indicating that out of all the instances predicted as positive by the model, approximately 91.98% of them are actually true positives.
- The recall of the Random Forest model is also around 0.9208, suggesting that the model is able to correctly identify approximately 92.08% of the positive instances in the test data



## 2-b. Gradient Boosting Classifier

**Initial Model**

In [11]:
from sklearn.ensemble import GradientBoostingClassifier

GB = GradientBoostingClassifier(max_depth=5,  n_estimators=100)

GB.fit(X_train, y_train)
prediction = GB.predict(X_test)



KeyboardInterrupt: 

In [ ]:
print("Test score: ",GB.score(X_test, y_test))
print("Training score: ", GB.score(X_train, y_train))

**Hyperparameter tuning with cross validation**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

# hyperparameter grid
param_grid = {
    'learning_rate': [0.1, 0.05, 0.01, 0.001],
    'max_depth': [3, 5, 7, 9, 11],
    'n_estimators': [50, 100, 150, 200, 250]
}


GB = GradientBoostingClassifier()
grid_search = GridSearchCV(GB, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# print the cross-validation scores
cv_results = grid_search.cv_results_
for mean_score, params in zip(cv_results["mean_test_score"], cv_results["params"]):
    print("Mean CV score:", mean_score, "Parameters:", params)


best_GB = grid_search.best_estimator_
best_cv_score = grid_search.best_score_

# predict the test 
prediction = best_GB.predict(X_test)


# print("Test score:", best_GB.score(X_test, y_test))
# print("Training score:", best_GB.score(X_train, y_train))
# print("Best parameters:", grid_search.best_params_)
# print("Best CV score:", best_score) # the mean cross-validation score of the best model.


In [ ]:
print("Test score:", best_GB.score(X_test, y_test))
print("Training score:", best_GB.score(X_train, y_train))
print("Best parameters:", grid_search.best_params_)
print("Best CV score:", best_cv_score) # the mean cross-validation score of the best model.


**Gradient Boost Final Model**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
GB = GradientBoostingClassifier(learning_rate=0.1, max_depth=9, n_estimators=250)

GB.fit(X_train, y_train)
prediction = GB.predict(X_test)
gb_accuracy_test = GB.score(X_test, y_test)
gb_accuracy_train = GB.score(X_train, y_train)

print("Test score: ", gb_accuracy_test)
print("Training score: ", gb_accuracy_train)


In [ ]:
## Check precision, recall, and F1 score
gb_precision = precision_score(y_test, prediction, average='weighted')
gb_recall = recall_score(y_test, prediction, average='weighted')
gb_f1 = f1_score(y_test, prediction, average='weighted')

print("Gradient Boosting Classifier Precision:", gb_precision)
print("Gradient Boosting Classifier Recall:", gb_recall)
print("Gradient Boosting Classifier F1 score:", gb_f1)


- The precision score of 0.9637 indicates that out of all instances predicted as positive by the model, approximately 96.37% of them are actually true positives. This indicates a high level of accuracy in predicting positive instances.

- The recall score of 0.9639 also suggests that the model correctly identifies approximately 96.39% of the positive instances present in the test data. This indicates a high level of sensitivity in detecting positive instances.

## 2-c. Nueral Network

In [ ]:
from sklearn.neural_network import MLPClassifier

MLP = MLPClassifier(solver='adam', activation='logistic', alpha=0.4, tol=1e-5,
                        hidden_layer_sizes=(100, 20), max_iter=500)
MLP.fit(X_train, y_train)
prediction = MLP.predict(X_test)


In [ ]:
print("Test score: ",MLP.score(X_test, y_test))
print("Training score: ", MLP.score(X_train, y_train))

**Hyperparameter tuning with cross validation**

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'solver': ['adam'],
    'activation': ['logistic', 'relu'],
    'alpha': [0.1, 0.2, 0.3],
    'hidden_layer_sizes': [(100,), (100, 50), (50, 20)],
    'max_iter': [200, 300, 400]
}


mlp = MLPClassifier()
grid_search = GridSearchCV(mlp, param_grid, cv=5)

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_


test_score = best_model.score(X_test, y_test)
train_score = best_model.score(X_train, y_train)

print("Test score:", test_score)
print("Training score:", train_score)
print("Best parameters:", best_params)


**Neural Network Final Model**

In [ ]:
from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier(solver='adam', activation='relu', alpha=0.2,
                        hidden_layer_sizes=(100, 50), max_iter=400)
MLP.fit(X_train, y_train)
prediction = MLP.predict(X_test)
mlp_accuracy_test = GB.score(X_test, y_test)
mlp_accuracy_train = GB.score(X_train, y_train)

print("Test score: ", mlp_accuracy_test)
print("Training score: ", mlp_accuracy_train)

In [ ]:
## Check precision and recall 
mlp_precision = precision_score(y_test, prediction, average='weighted')
mlp_recall = recall_score(y_test, prediction, average='weighted')
mlp_f1 = f1_score(y_test, prediction, average='weighted')

print("Nerual Network Precision:", mlp_precision)
print("Nerual Network Recall:", mlp_recall)
print("Nerual Network F1 score:", mlp_f1)


- The precision score of 0.7391 suggests that out of all instances predicted as positive by the model, approximately 73.91% of them are actually true positives. This indicates the model's ability to correctly identify positive instances while minimizing false positives.

- The recall score of 0.8296 suggests that the model correctly identifies approximately 82.96% of the positive instances present in the test data. This indicates the model's ability to capture a large portion of the positive instances and avoid false negatives.


## 2-d. XGBoost

In [12]:
import xgboost as xgb

first_XBG = xgb.XGBClassifier()

first_XBG.fit(X_train, y_train)
first_prediction = first_XBG.predict(X_test)

xgb_accuracy_test = first_XBG.score(X_test, y_test)
xgb_accuracy_train = first_XBG.score(X_train, y_train)

In [13]:
print("Test score: ", xgb_accuracy_test)
print("Training score: ", xgb_accuracy_train)

Test score:  0.9667870036101083
Training score:  1.0


**Hyperparameter tuning with cross validation**

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
param_grid = {
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.01, 0.001]
}


XBG = xgb.XGBClassifier()

grid_search = GridSearchCV(XBG, param_grid, cv=5)

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

test_score = best_model.score(X_test, y_test)
train_score = best_model.score(X_train, y_train)

print("Test score:", test_score)
print("Training score:", train_score)
print("Best parameters:", best_params)


Since the test score is very close to the test score from out initial model, we decided to choose the initial model for simplicity reason.

In [ ]:
from sklearn.metrics import precision_score, recall_score

xgb_precision = precision_score(y_test, first_prediction, average='weighted')
xgb_recall = recall_score(y_test, first_prediction, average='weighted')
xgb_f1 = f1_score(y_test, first_prediction, average = 'weighted')

print("XGBoost Classifier Precision:", xgb_precision)
print("XGBoost Classifier Recall:", xgb_recall)
print("XGBoost Classifier F1 score:", xgb_f1)


# 3. Model Comparison

In [ ]:
import matplotlib.pyplot as plt

# Define the models and their corresponding performance metrics
models = ['RandomForest', 'GradientBoosting', 'MLP', 'XGBoost']
accuracies = [rf_accuracy_test, gb_accuracy_test, mlp_accuracy_test, xgb_accuracy_test]
precisions = [rf_precision, gb_precision, mlp_precision, xgb_precision]
recalls = [rf_recall, gb_recall, mlp_recall, xgb_recall]
f1_scores = [rf_f1, gb_f1, mlp_f1, xgb_f1]

# Plot the metrics
plt.figure(figsize=(10, 6))

# Accuracy
plt.subplot(221)
plt.bar(models, accuracies)
plt.title('Accuracy')
plt.ylim(0, 1)

# Precision
plt.subplot(222)
plt.bar(models, precisions)
plt.title('Precision')
plt.ylim(0, 1)

# Recall
plt.subplot(223)
plt.bar(models, recalls)
plt.title('Recall')
plt.ylim(0, 1)

# F1-score
plt.subplot(224)
plt.bar(models, f1_scores)
plt.title('F1-score')
plt.ylim(0, 1)

plt.tight_layout()
plt.show()


According to the above bar charts, gradient boosting classifier and XGBoost are the top 2 models with the best metrics. Although both Gradient Boosting and XGBoost show excellent performance, XGBoost is preferred since its initial model was picked. As a result, we chose the XGBoost model because to its performance, efficiency, and simplicity. 

# 4. Saving our best model 

In [14]:
import pickle  #Initialize the flask App
# Saving model to current directory
# Pickle serializes objects so they can be saved to a file, and loaded in a program again later on.
pickle.dump(first_XBG, open('model.pkl','wb'))
